In [20]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
import sys
import sklearn
import scipy
import math
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)

In [12]:
# load the file with all the data for the stars
a = pd.read_csv('../data/lc_data_new.txt')
a = a.to_numpy()

In [4]:
# the time points of all the data points in order

times = np.asarray([54945.74206,54945.8670833,54946.0182163,54946.2335259,54946.3377372,54946.5495655,54946.7334679,
         54947.1662509,55062.8253608,55091.0055606,55123.0864583,55153.9797114,55182.0366329,55215.9548927,
         55216.0352649,55245.7660019,55274.7398619,55307.5350333,55336.4281441,55370.695297,55399.0571196,
         55430.8109392,55461.8291274,55492.8064711,55522.7621353,55552.0843583,55585.5760016,55614.7389024,
         55677.4444762,55706.6440223,55738.4591443,55769.477399,55801.7624176,55832.8010616,55864.8001316,
         55895.757043,55930.8619526,55958.4268791,55986.5230115,56014.5579184,56047.5173675,56077.4525238,
         56105.5895974,56137.5273556,56168.8315615,56203.8547584,56236.834641,56267.9141396,56303.6729488,
         56330.563549,56357.495041,56390.4952381]) - 54833

In [5]:
# a model for a sine curve
def func(x, a, b, c, d):
    return a*np.sin(b * x + c) + d 

# a model for a straight line
def straightLine(x, m, b):
    return m*x + b

# a model for a sinecurve that is tiled on an axis
def func_tilt(x, a, b, c, d, e):
    return a*np.sin(b * x + c) + d + e*x

# a model for a straight line
def flatLine(x, b):
    return b

In [6]:
# given the parameters of a light curve and its corresponding default sinecurve fit
# calculates the error of the fit
# returns
# error_l1: the l1 error of the fit
# error_l2: the l2 error of the fit
def calculate_error(fn, times, params, flux, yerr):
    error_l1 = np.sum(np.abs(fn(times, *params) - flux)/yerr)
    error_l2 = np.sum( ((fn(times, *params) - flux) ** 2 )/ (yerr ** 2 ))
        
    return error_l1, error_l2

In [7]:
# given the data points of the flux of the starts, forcefully calculates the best straight line fit
# best sinecurve fit is defined by finding which curve returns the smallest error 
# uses the function calulate_error_line
# returns
# params: the parameters of the best straight line fit
# min_error_l1: the error of the above curve 
# min_error_l2: the error of the above curve 
def fit_line(fn, flux, times, yerr):
    
    params, pcov = curve_fit(fn, times, flux, maxfev=50000)
    error_l1, error_l2 = calculate_error(fn, times, params, flux, yerr)
    
    return error_l1, error_l2, params

In [8]:
# given the data points of the flux of the starts, forcefully calculates the best titled sine curve fit
# best sinecurve fit is defined by finding which curve returns the smallest error 
# uses the function calulate_error_tilt
# returns
# best_params_l1: the parameters of the sinecurve that fits the best using the l1 error
# min_error_l1: the error of the above curve 
# best_params_l2: the parameters of the sinecurve that fits the best using the l2 error
# min_error_l2: the error of the above curve 
def fit_curve(fn, flux, times, yerr):
    min_error_l1 = sys.maxsize
    best_params_l1 = [0] * 4
    best_params_l2 = [0] * 4
    min_error_l2 = sys.maxsize
    for n in np.linspace(0,0.1,100):
        if fn == func:
            try:
                params, pcov = curve_fit(fn, times, flux, p0 = [0.01, n, 0, 1], maxfev=50000)
            except RuntimeError:
                continue
        elif fn == func_tilt:
            try:
                params, pcov = curve_fit(fn, times, flux, p0 = [0.01, n, 0, 1, 0.001], maxfev=50000)
            except RuntimeError:
                continue
        error_l1, error_l2 = calculate_error(fn, times, params, flux, yerr)
        if error_l1 < min_error_l1:
            min_error_l1 = error_l1
            best_params_l1 = params
            
        if error_l2 < min_error_l2:
            min_error_l2 = error_l2
            best_params_l2 = params
    
    return best_params_l1, min_error_l1, best_params_l2, min_error_l2

In [ ]:
# dictionary to store data
data = {'KIC': [], 
        'curveErrorL1': [],
        'curveErrorL2': [],
        'straightLineErrorL1': [],
        'straightLineErrorL2': [],
        'curtilErrorL1': [],
        'curtilErrorL2': [],
        'flatLineErrorL1': [],
       'flatLineErrorL2': [],}

params = {'KIC': [], 
        'curveParamL1': [],
        'curveParamL2': [],
        'straightLineParam': [],
        'curtilParamL1': [],
        'curtilParamL2': [],
        'flatLineParam': []}

#loop to go through all the stars
for i in tqdm(range(len(a))):
    id = a[i][0]
    flux = a[a[:,0] == id][0][1:53]
    yerr = a[a[:,0] == id][0][57:]
    if not np.isfinite(flux).all():
        continue
    if 
    
    curve_p_l1, curve_e_l1, curve_p_l2, curve_e_l2 = fit_curve(func, flux, times, yerr)
    straight_e_l1, straight_e_l2, straight_p = fit_line(straightLine, flux, times, yerr)
    curtil_p_l1, curtil_e_l1, curtil_p_l2, curtil_e_l2 = fit_curve(func_tilt, flux, times, yerr)
    flat_e_l1, flat_e_l2, flat_p = fit_line(flatLine, flux, times, yerr)
    
    data['KIC'].append(id)
    data['curveErrorL1'].append(curve_e_l1)
    data['curveErrorL2'].append(curve_e_l2)
    data['straightLineErrorL1'].append(straight_e_l1)
    data['straightLineErrorL2'].append(straight_e_l2)
    data['curtilErrorL1'].append(curtil_e_l1)
    data['curtilErrorL2'].append(curtil_e_l2)
    data['flatLineErrorL1'].append(flat_e_l1)
    data['flatLineErrorL2'].append(flat_e_l2)
    
    params['KIC'].append(id)
    params['curveParamL1'].append(curve_p_l1)
    params['curveParamL2'].append(curve_p_l2)
    params['straightLineParam'].append(straight_p)
    params['curtilParamL1'].append(curtil_p_l1)
    params['curtilParamL2'].append(curtil_p_l2)
    params['flatLineParam'].append(flat_p)

  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
# ran in 4 parts on multi-core processor, outputted 4 separate files for each csv

df = pd.DataFrame(data, columns = ['KIC', 'curveErrorL1', 'curveErrorL2', 
                                   'straightLineErrorL1', 'straightLineErrorL2',
                                   'curtilErrorL1', 'curtilErrorL2', 'flatLineErrorL1',
                                   'flatLineErrorL2'])

df.to_csv("../output/errors.csv", index=False)

params_df = pd.DataFrame(params, columns = ['KIC', 'curveParamL1', 'curveParamL2', 
                                         'straightLineParam', 'curtilParamL1', 'curtilParamL2',
                                         'flatLineParam'])
params_df.to_csv("../output/params.csv", index=False)